In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# to bypass warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_path = "/home/dcti-02-11/projects/data/NYC_Taxi_Trips/data/2020_taxi_trips.csv"

trips_2020 = pd.read_csv(file_path)

In [3]:
trips_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734051 entries, 0 to 1734050
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   lpep_pickup_datetime   object 
 2   lpep_dropoff_datetime  object 
 3   store_and_fwd_flag     object 
 4   RatecodeID             float64
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   passenger_count        float64
 8   trip_distance          float64
 9   fare_amount            float64
 10  extra                  float64
 11  mta_tax                float64
 12  tip_amount             float64
 13  tolls_amount           float64
 14  improvement_surcharge  float64
 15  total_amount           float64
 16  payment_type           float64
 17  trip_type              float64
 18  congestion_surcharge   float64
dtypes: float64(14), int64(2), object(3)
memory usage: 251.4+ MB


In [4]:
trips_2020 = trips_2020.dropna()

In [5]:
trips_2020.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-01-01 06:47:28.000,2020-01-01 06:52:54.000,N,1.0,74,75,1.0,1.47,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
1,2.0,2020-01-01 13:25:34.000,2020-01-01 13:30:43.000,N,1.0,74,75,1.0,1.49,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
2,2.0,2020-01-01 14:20:35.000,2020-01-01 14:26:25.000,N,1.0,74,75,1.0,1.31,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
3,2.0,2020-01-02 06:56:47.000,2020-01-02 07:03:03.000,N,1.0,74,75,1.0,1.43,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
4,2.0,2020-01-02 09:34:46.000,2020-01-02 09:41:02.000,N,1.0,74,75,1.0,1.10,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0


## Replace and convert columns to the appropriate data types

In [6]:
# convert passenger count from float to int
list_convert = ['VendorID','RatecodeID','payment_type','trip_type','passenger_count']
trips_2020[list_convert] = trips_2020[list_convert].astype(int)

In [7]:
# Replace Y and N with True and False respectively
trips_2020['store_and_fwd_flag'] = trips_2020['store_and_fwd_flag'].replace({'N':False, 'Y':True})

In [8]:
# convert the following columns to category
list_convert = ['VendorID','RatecodeID','payment_type','trip_type']
trips_2020[list_convert] = trips_2020[list_convert].astype('category')

In [9]:
# convert pickup and dropoff to datetime
trips_2020['lpep_pickup_datetime'] = pd.to_datetime(trips_2020['lpep_pickup_datetime'])
trips_2020['lpep_dropoff_datetime'] = pd.to_datetime(trips_2020['lpep_dropoff_datetime'])

## Adding new Columns

In [10]:
# convert distance from miles to km
trips_2020['trip_distance_km'] = trips_2020['trip_distance'] * 1.609344

In [11]:
# get time difference from dropoff and pickup
trips_2020['time_diff'] = trips_2020['lpep_dropoff_datetime'] - trips_2020['lpep_pickup_datetime']

In [12]:
# get the pickup and dropoffs dates only
trips_2020['pickup_date'] = pd.to_datetime(trips_2020['lpep_pickup_datetime'].dt.date)
trips_2020['dropoff_date'] = pd.to_datetime(trips_2020['lpep_dropoff_datetime'].dt.date)

In [13]:
# add columns for month of the year
trips_2020['pickup_month'] = trips_2020['pickup_date'].dt.month
trips_2020['dropoff_month'] = trips_2020['dropoff_date'].dt.month

In [14]:
# add columns for day of the week
trips_2020['pickup_day'] = trips_2020['pickup_date'].dt.day_name()
trips_2020['dropoff_day'] = trips_2020['dropoff_date'].dt.day_name()

In [15]:
# Extract the hours, minutes, and seconds from the timedelta column
trips_2020['Hours'] = trips_2020['time_diff'].dt.components.hours
trips_2020['Minutes'] = trips_2020['time_diff'].dt.components.minutes
trips_2020['Seconds'] = trips_2020['time_diff'].dt.components.seconds

# Create a new column with the time as a string (hh:mm:ss format)
trips_2020['TimeOnly'] = trips_2020.apply(lambda row: f"{row['Hours']:02}:{row['Minutes']:02}:{row['Seconds']:02}", axis=1)

trips_2020['TimeOnly'] = pd.to_timedelta(trips_2020['TimeOnly']).dt.total_seconds().astype('timedelta64[s]')

# Extract the time portion (hours, minutes, seconds) from the timedelta column
trips_2020['Hours'] = trips_2020['time_diff'].dt.components.hours
trips_2020['Minutes'] = trips_2020['time_diff'].dt.components.minutes
trips_2020['Seconds'] = trips_2020['time_diff'].dt.components.seconds

# Calculate the total seconds representing the time
trips_2020['TotalSeconds'] = trips_2020['Hours'] * 3600 + trips_2020['Minutes'] * 60 + trips_2020['Seconds']

# Convert the total seconds to a new timedelta column
trips_2020['TimeOnlyy'] = pd.to_timedelta(trips_2020['TotalSeconds'], unit='s')

# Drop unnecessary columns
trips_2020 = trips_2020.drop(['Hours', 'Minutes', 'Seconds', 'TotalSeconds'], axis=1)

# Rearrange the columns in the desired order
df = df[['TimeDeltaColumn', 'TimeOnly', 'Hours', 'Minutes', 'Seconds']]

## Remove negative distances and negative days

In [31]:
# remove rows with trip distance less that 0
trips_2020 = trips_2020[trips_2020['trip_distance_km'] > 0]

In [32]:
# remove negative time difference
trips_2020 = trips_2020[trips_2020['time_diff'] > pd.Timedelta(0)]

In [19]:
trips_2020['fare_amount'].value_counts()

6.00     61380
6.50     58843
7.00     57552
5.50     56884
5.00     53102
         ...  
14.85        1
22.57        1
53.42        1
53.82        1
82.03        1
Name: fare_amount, Length: 1841, dtype: int64

In [ ]:
trips_2020.trip_distance_km.value_counts()

In [38]:
trips_2020['time_diff'].max()

Timedelta('0 days 23:59:58')

In [16]:
trips_2020.tail()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,pickup_date,dropoff_date,pickup_month,dropoff_month,pickup_day,dropoff_day,Hours,Minutes,Seconds,TimeOnly
1730571,1,2020-02-17 09:16:54,2020-02-17 09:58:34,False,1,220,162,2,0.0,30.20,...,2020-02-17,2020-02-17,2,2,Monday,Monday,0,41,40,00:41:40
1730572,1,2020-02-18 08:51:38,2020-02-18 09:34:19,False,1,200,75,2,0.0,30.20,...,2020-02-18,2020-02-18,2,2,Tuesday,Tuesday,0,42,41,00:42:41
1730573,1,2020-02-22 08:52:44,2020-02-22 09:13:19,False,1,220,48,2,0.0,28.20,...,2020-02-22,2020-02-22,2,2,Saturday,Saturday,0,20,35,00:20:35
1730574,2,2020-06-22 16:25:45,2020-06-22 16:26:13,False,5,215,215,0,0.0,-43.56,...,2020-06-22,2020-06-22,6,6,Monday,Monday,0,0,28,00:00:28
1730575,1,2020-09-05 09:01:04,2020-09-05 09:01:04,True,5,93,264,2,0.0,0.00,...,2020-09-05,2020-09-05,9,9,Saturday,Saturday,0,0,0,00:00:00


In [17]:
trips_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205954 entries, 0 to 1730575
Data columns (total 31 columns):
 #   Column                 Non-Null Count    Dtype          
---  ------                 --------------    -----          
 0   VendorID               1205954 non-null  category       
 1   lpep_pickup_datetime   1205954 non-null  datetime64[ns] 
 2   lpep_dropoff_datetime  1205954 non-null  datetime64[ns] 
 3   store_and_fwd_flag     1205954 non-null  bool           
 4   RatecodeID             1205954 non-null  category       
 5   PULocationID           1205954 non-null  int64          
 6   DOLocationID           1205954 non-null  int64          
 7   passenger_count        1205954 non-null  int64          
 8   trip_distance          1205954 non-null  float64        
 9   fare_amount            1205954 non-null  float64        
 10  extra                  1205954 non-null  float64        
 11  mta_tax                1205954 non-null  float64        
 12  tip_amount    

In [ ]:
trips_2020.isna().mean() * 100

In [45]:
trips_2020.groupby(['pickup_day']).count()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,payment_type,trip_type,congestion_surcharge,trip_distance_km,time_diff,pickup_date,dropoff_date,pickup_month,dropoff_month,dropoff_day
pickup_day,,,,,,,,,,,,,,,,,,,,,
Friday,188044,188044,188044,188044,188044,188044,188044,188044,188044,188044,...,188044,188044,188044,188044,188044,188044,188044,188044,188044,188044
Monday,152201,152201,152201,152201,152201,152201,152201,152201,152201,152201,...,152201,152201,152201,152201,152201,152201,152201,152201,152201,152201
Saturday,167063,167063,167063,167063,167063,167063,167063,167063,167063,167063,...,167063,167063,167063,167063,167063,167063,167063,167063,167063,167063
Sunday,129712,129712,129712,129712,129712,129712,129712,129712,129712,129712,...,129712,129712,129712,129712,129712,129712,129712,129712,129712,129712
Thursday,179228,179228,179228,179228,179228,179228,179228,179228,179228,179228,...,179228,179228,179228,179228,179228,179228,179228,179228,179228,179228
Tuesday,158021,158021,158021,158021,158021,158021,158021,158021,158021,158021,...,158021,158021,158021,158021,158021,158021,158021,158021,158021,158021
Wednesday,172297,172297,172297,172297,172297,172297,172297,172297,172297,172297,...,172297,172297,172297,172297,172297,172297,172297,172297,172297,172297


In [46]:
trips_2020.groupby(['pickup_day']).sum()

,store_and_fwd_flag,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_distance_km,pickup_month,dropoff_month
pickup_day,,,,,,,,,,,,,,,,
Friday,629,18820034,24013929,243523,558818.76,2326032.50,90057.50,91546.00,234250.18,33718.63,55788.3,2909285.51,87510.75,8.993316e+05,701932,702100
Monday,489,15378125,19881830,196703,517113.43,1903194.57,66537.75,74095.50,190920.65,29724.05,45138.6,2381657.92,81443.75,8.322134e+05,624647,624669
Saturday,508,16814222,21333662,217613,530732.38,2074780.18,38125.00,81021.60,203444.49,27587.40,49494.9,2545071.92,78390.75,8.541310e+05,632670,632865
Sunday,386,13143679,16755656,170797,400339.52,1606442.89,30563.26,62876.05,164318.10,22611.45,38455.5,1983173.80,64805.50,6.442840e+05,481420,481429
Thursday,577,18065228,23176846,233665,520409.85,2218944.33,84622.00,87262.00,230534.81,32772.43,53173.8,2789931.22,92823.25,8.375185e+05,684092,683988
Tuesday,486,15867832,20582638,205137,468059.86,1976475.32,72546.25,77073.65,201934.87,30383.95,46892.4,2481107.84,85612.00,7.532693e+05,648246,648158
Wednesday,551,17263655,22357098,224268,755252.70,2164403.90,79347.00,83839.50,223383.06,33575.22,51110.1,2717573.73,91526.50,1.215461e+06,678787,678804


In [54]:
trips_2020.groupby(['pickup_month'])['total_amount', 'tip_amount','congestion_surcharge'].max()

,total_amount,tip_amount,congestion_surcharge
pickup_month,,,
1,753.80,450.00,2.75
2,651.50,641.20,2.75
3,462.27,449.97,2.75
4,398.76,100.00,2.75
5,655.30,160.00,2.75
6,498.80,480.00,2.75
7,803.80,88.00,2.75
8,550.80,200.00,2.75
9,537.85,89.06,2.75
